In [1]:
from datasets import load_dataset
from utils import jpeg_compress, webp_compress, pad, crop, nn_compress, hific_lo_compress
import compressai
import evaluate
from transformers import AutoFeatureExtractor, SegformerForSemanticSegmentation
# from transformers im+port SegformerImageProcessor, SegformerForSemanticSegmentation
# from evaluate import evaluator
# from transformers import pipeline
# import torch

2023-10-16 12:30:44.937979: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-16 12:30:44.958166: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-16 12:30:44.958188: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-16 12:30:44.958206: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-16 12:30:44.962587: I tensorflow/core/platform/cpu_feature_g

In [2]:
def jpeg_compress_ade(sample):
    img,bpp = jpeg_compress(sample['image'])
    sample['image'] = img
    sample['bpp' ] = bpp
    return sample
    
def webp_compress_ade(sample):
    img,bpp = webp_compress(sample['image'])
    sample['image'] = img
    sample['bpp' ] = bpp
    return sample

device = "cuda"
net_mbt2018 = compressai.zoo.mbt2018(quality=1, pretrained=True).eval().to(device)
def mbt2018_compress_ade(sample):
    img,bpp = nn_compress(sample['image'],net_mbt2018,device)
    sample['image'] = img
    sample['bpp'] = bpp
    return sample

def hific_lo_compress_ade(sample):
    img,bpp = hific_lo_compress(sample['image'])
    sample['image'] = img
    sample['bpp' ] = bpp
    return sample

In [3]:
image_compression_methods = [
    jpeg_compress_ade,
    webp_compress_ade,
    mbt2018_compress_ade,
    hific_lo_compress_ade
]
models = [
    "nvidia/segformer-b0-finetuned-ade-512-512",
    "nvidia/segformer-b2-finetuned-ade-512-512",
    "nvidia/segformer-b5-finetuned-ade-640-640"
]

In [4]:
ade20k = [load_dataset("scene_parse_150",split='validation[:100]')]

In [5]:
for method in image_compression_methods:
    ade20k.append(ade20k[0].map(method))

In [6]:
mean_iou = evaluate.load("mean_iou")

In [13]:
from torchvision import transforms
import numpy as np
def evaluate_segmentation_miou(data):
    x = []
    y = []
    for sample in data:
        ground_truth = sample['annotation']
        img = sample['image']
        inputs = extractor(images=img, return_tensors="pt",do_resize=False).to("cuda:0")
        outputs = model(**inputs)
        logits = outputs.logits  # shape (batch_size, num_labels, height/4, width/4)
        predictions = logits.argmax(dim=1)[0].detach().cpu()
        resize = transforms.Resize(
            (ground_truth.height,ground_truth.width),
            antialias=True
        )
        predictions = resize(
            predictions.unsqueeze(0).unsqueeze(0)
        )[0][0]
    
        x.append(np.array(ground_truth))
        y.append(np.array(predictions))
    return mean_iou.compute(
        predictions=y,
        references=x,
        num_labels=150,
        ignore_index=255,
        reduce_labels=True
    )['mean_iou']

In [14]:
img = ade20k[0][0]['image']
inputs = extractor(images=img, return_tensors="pt",do_resize=False).to("cuda:0")
outputs = model(**inputs)
logits = outputs.logits  # shape (batch_size, num_labels, height/4, width/4)
predictions = logits.argmax(dim=1)[0].detach()

In [17]:
eval_results = []
for i_method,method in enumerate(ade20k):
    eval_results.append([])
    for i_model,model_name in enumerate(models):
        extractor = AutoFeatureExtractor.from_pretrained(model_name)
        model = SegformerForSemanticSegmentation.from_pretrained(model_name).to("cuda:0")
        eval_results[i_method].append(
            evaluate_segmentation_miou(method)
        )

/home/dan/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label


/home/dan/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:259: RuntimeWarning: invalid value encountered in divide
  iou = total_area_intersect / total_area_union


In [18]:
eval_results

[[0.2179962219278336, 0.2661512252963287, 0.28154993702999215],
 [0.09736559130225683, 0.15950990338963633, 0.16076962220495397],
 [0.14695622531768182, 0.18583407586326606, 0.20174996631311962],
 [0.11058525161913754, 0.13802783140493485, 0.1540462591520832],
 [0.20660080180962445, 0.24499494687817394, 0.2615260820411224]]